# 0.0 IMPORTS

## 01. Libraries

In [56]:
import warnings
import inflection

import dask.dataframe as dd

from dask.distributed import Client, progress

In [21]:
# import datetime
# import numpy as np
# import scipy.stats  as ss

# from IPython.display         import Image
# from IPython.core.display    import HTML

# import matplotlib.pyplot as plt
# import seaborn as sns

# 0.2 Notebook Config

In [57]:
warnings.filterwarnings("ignore")

In [58]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:61518 Dashboard: http://127.0.0.1:61517/status,Cluster Workers: 4 Cores: 12 Memory: 16.98 GB


In [22]:
# def jupyter_settings():
#     %matplotlib inline
    
#     plt.style.use('bmh')
#     plt.rcParams['figure.figsize'] = [25, 12]
#     plt.rcParams['font.size'] = 24
    
#     display( HTML('<style>.container { width:100% !important; }</style>'))
    
#     warnings.filterwarnings("ignore")
    
#     sns.set()

In [23]:
# seed = 42
# np.random.seed(seed)

# jupyter_settings()

## 0.3 Helper Functions

## 0.4 Load Data

### 0.4.1 Load Raw Data

In [6]:
data_train_raw = dd.read_csv('../00-Data/csv_data/train.csv')
data_producto_raw = dd.read_csv('../00-Data/csv_data/producto_tabla.csv')
data_cliente_raw = dd.read_csv('../00-Data/csv_data/cliente_tabla.csv')
data_town_state_raw = dd.read_csv('../00-Data/csv_data/town_state.csv')

Wall time: 103 ms


### 0.4.2 Merge Dataset

In [8]:
data_raw = dd.merge(data_train_raw, data_producto_raw, how='left', on='Producto_ID')
data_raw = dd.merge(data_raw, data_cliente_raw, how='left', on='Cliente_ID')
data_raw = dd.merge(data_raw, data_town_state_raw, how='left', on='Agencia_ID')
data_raw = data_raw.drop(['Producto_ID', 'Cliente_ID', 'Agencia_ID'], axis=1)
data_raw.head()

Wall time: 2.24 s


,Semana,Canal_ID,Ruta_SAK,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,NombreProducto,NombreCliente,Town,State
0,3,7,3301,3,25.14,0,0.0,3,Roles Canela 2p 120g BIM 1212,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,3,7,3301,4,33.52,0,0.0,4,Roles Glass 2p 135g BIM 1216,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
2,3,7,3301,4,39.32,0,0.0,4,Panquecito Gota Choc 2p 140g BIM 1238,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
3,3,7,3301,4,33.52,0,0.0,4,Mantecadas Vainilla 4p 125g BIM 1240,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,3,7,3301,3,22.92,0,0.0,3,Donitas Espolvoreadas 6p 105g BIM 1242,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."


# 1.0 DATA DESCRIPTION

In [9]:
df01 = data_raw.copy()

## 1.1 Columns

In [10]:
df01.columns

Index(['Semana', 'Canal_ID', 'Ruta_SAK', 'Venta_uni_hoy', 'Venta_hoy',
       'Dev_uni_proxima', 'Dev_proxima', 'Demanda_uni_equil', 'NombreProducto',
       'NombreCliente', 'Town', 'State'],
      dtype='object')

### 1.2.1 Rename Columns

In [13]:
snakecase = lambda col: inflection.underscore(col)

new_columns = list(map(snakecase, df01.columns))

# rename
df01.columns = new_columns

### 1.2.2 New Name Columns

In [15]:
df01.columns

Index(['semana', 'canal_id', 'ruta_sak', 'venta_uni_hoy', 'venta_hoy',
       'dev_uni_proxima', 'dev_proxima', 'demanda_uni_equil',
       'nombre_producto', 'nombre_cliente', 'town', 'state'],
      dtype='object')

## 1.3 Data Dimensions

In [19]:
print(f'Number of Rows: {df01.shape[0].compute()}')
print(f'Number of Columns: {df01.shape[1]}')

Number of Rows: 74773833
Number of Columns: 12


## 1.4 Data Types

In [20]:
df01.dtypes

semana                 int64
canal_id               int64
ruta_sak               int64
venta_uni_hoy          int64
venta_hoy            float64
dev_uni_proxima        int64
dev_proxima          float64
demanda_uni_equil      int64
nombre_producto       object
nombre_cliente        object
town                  object
state                 object
dtype: object

## 1.5 Missing Values

In [54]:
data_raw.isnull().sum().compute()

Wall time: 1min 29s


Semana               0
Canal_ID             0
Ruta_SAK             0
Venta_uni_hoy        0
Venta_hoy            0
Dev_uni_proxima      0
Dev_proxima          0
Demanda_uni_equil    0
NombreProducto       0
NombreCliente        0
Town                 0
State                0
dtype: int64

### 1.5.2 Fillout NA

There is no NA values

## 1.6 Descriptive Statistical

In [22]:
num_attributes = df01.select_dtypes(include=['int64', 'float64'])

In [89]:
describe = num_attributes.describe().compute().T

describe['Relative Std'] = describe['std'] / describe['mean']
describe['range'] = describe['max'] - describe['min']
describe['IQR'] = describe['75%'] - describe['25%']
describe = describe[['min', 'max', 'range', 'mean','std', '50%', 'IQR', '25%', '75%', 'Relative Std']]
describe.columns = ['Min', 'Max', 'Range', 'Mean', 'Std', 'Median', 'IQR', '25%', '75%', 'Relative Std']
describe

,Min,Max,Range,Mean,Std,Median,IQR,25%,75%,Relative Std
semana,3.0,9.0,6.0,5.950119,2.013151,6.00,4.00,4.00,8.00,0.338338
canal_id,1.0,11.0,10.0,1.388785,1.470505,1.00,0.00,1.00,1.00,1.058843
ruta_sak,1.0,9991.0,9990.0,2120.527163,1494.363905,1614.00,1691.00,1213.00,2904.00,0.704713
venta_uni_hoy,0.0,7200.0,7200.0,7.340670,22.165330,4.00,6.00,2.00,8.00,3.019524
venta_hoy,0.0,647360.0,647360.0,68.919069,340.325673,34.59,45.31,18.86,64.17,4.938048
dev_uni_proxima,0.0,250000.0,250000.0,0.129932,29.207650,0.00,0.00,0.00,0.00,224.792458
dev_proxima,0.0,130760.0,130760.0,1.240560,39.131953,0.00,0.00,0.00,0.00,31.543777
demanda_uni_equil,0.0,5000.0,5000.0,7.255290,21.972330,4.00,6.00,2.00,8.00,3.028457
